<a href="https://colab.research.google.com/github/muglikar/youtubeVid2Drive/blob/master/YoutubeNgrokServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
#      YOUTUBE DOWNLOADER - V29 (Final with Automatic Keep-Alive)
# ==============================================================================
# This is the final, complete, and feature-rich application. It includes an
# automatic background thread that pings the Colab log every 15 minutes to
# prevent the session from timing out due to inactivity, removing the need
# for a manual UI button.
# ==============================================================================

# ----------------------------
# SETUP, IMPORTS & CONFIG
# ----------------------------
print("🚀 Starting setup...")
!pip install -q flask flask-session pyngrok yt-dlp google-api-python-client google-auth-httplib2 google-auth-oauthlib

import os
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

import json, uuid, time, subprocess, io
from threading import Thread
from datetime import timedelta, datetime
from flask import Flask, request, render_template_string, jsonify, redirect, url_for, session
from flask_session import Session
from pyngrok import ngrok
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import yt_dlp

print("✅ All packages installed and imported.")

# --- Flask App Configuration ---
app = Flask(__name__)
app.config['SECRET_KEY'] = os.urandom(24)
app.config['SESSION_TYPE'] = 'filesystem'
Session(app)

# --- ‼️‼️ ACTION REQUIRED ‼️‼️ ---
NGROK_STATIC_DOMAIN = "YOUR_STATIC_DOMAIN_HERE.ngrok-free.app"
NGROK_AUTH_TOKEN = "<YOUR_NGROK_AUTH_TOKEN>"
# ------------------------------------

if NGROK_AUTH_TOKEN == "<YOUR_NGROK_AUTH_TOKEN>" or NGROK_STATIC_DOMAIN == "YOUR_STATIC_DOMAIN_HERE.ngrok-free.app":
    print("🛑 WARNING: Please set your NGROK_AUTH_TOKEN and NGROK_STATIC_DOMAIN variables!")

# --- Global State ---
DOWNLOAD_TASKS = {}
TEMP_DOWNLOAD_DIR = "/content/temp_downloads"
os.makedirs(TEMP_DOWNLOAD_DIR, exist_ok=True)
REDIRECT_URI = f"https://{NGROK_STATIC_DOMAIN}/oauth2callback"

if not os.path.exists('client_secrets.json'):
    print("‼️ ERROR: `client_secrets.json` not found. Please upload it.")
else:
    print("✅ `client_secrets.json` found.")

SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/userinfo.email', 'openid']

# ----------------------------
# WEB APP UI (Keep-Alive section removed)
# ----------------------------
HTML_TEMPLATE = """
<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>YouTube Downloader</title><style>:root{--primary-color:#4a90e2;--secondary-color:#f5f5f5;--font-color:#333;--border-color:#ddd;--success-color:#4CAF50;--error-color:#f44336}body{font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,sans-serif;background-color:var(--secondary-color);color:var(--font-color);margin:0;padding:20px;display:flex;justify-content:center;align-items:center;min-height:100vh}.container{width:100%;max-width:600px;background:white;padding:25px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,.1);text-align:center}h1,h3{color:var(--primary-color)}.form-group{margin-bottom:15px;text-align:left}label{display:block;margin-bottom:5px;font-weight:700}input,select,button{width:100%;padding:12px;margin-top:5px;border-radius:5px;border:1px solid var(--border-color);box-sizing:border-box;font-size:16px}button{background-color:var(--primary-color);color:#fff;border:none;cursor:pointer}button:disabled{background-color:#a0a0a0;cursor:not-allowed}.login-btn{background-color:var(--success-color);padding:15px;font-size:18px}.user-info{text-align:right;margin-bottom:10px;font-size:12px}.status{padding:10px;border-radius:5px;margin-top:15px;font-weight:700}.status-info{background-color:#e7f3fe;color:#31708f}.status-success{background-color:#dff0d8;color:#3c763d}.status-error{background-color:#f2dede;color:#a94442}#info-preview,#progress-container{margin-top:20px;padding:15px;border-radius:5px;display:none}#info-preview{border:1px solid var(--border-color);display:flex;align-items:center}#info-preview img{max-width:120px;border-radius:5px;margin-right:15px}#history-list{list-style-type:none;padding:0;text-align:left;max-height:200px;overflow-y:auto;border:1px solid #eee;padding:10px}</style></head><body><div class="container"><h1>📥 YouTube Downloader</h1>{% if 'credentials' in session %}<div class="user-info">Logged in as: {{ session['email'] }} <br><a href="/logout">Logout</a></div><div id="main-content"><div class="form-group"><label for="link">YouTube URL</label> <input type="text" id="link" placeholder="Enter a video or playlist URL"></div><button onclick="fetchInfo()" id="fetch-btn">Fetch Info</button><div id="info-preview"><img id="thumbnail" src="" alt="Video thumbnail"><div id="info-preview-text"><strong id="video-title"></strong><p id="video-duration"></p></div></div><div class="form-group"><label for="mode">Download Mode</label> <select id="mode" onchange="modeChanged()"><option value="video">Video</option><option value="audio">Audio</option><option value="transcript">Transcript</option></select></div><div class="form-group" id="format-select-group" style="display:none"><label for="format_id">Select Format/Quality</label> <select id="format_id"></select></div><div class="form-group" id="transcript-select-group" style="display:none"><label for="transcript_lang">Select Transcript Language</label> <select id="transcript_lang"></select></div><div class="form-group"><label for="folder">Google Drive Folder Name</label> <input type="text" id="folder" placeholder="e.g., 'My Downloads' (optional)"></div><button onclick="startDownload()" id="download-btn" disabled>Download</button></div><div id="progress-container"><p id="progress-status-text">Starting...</p><progress id="progress-bar" value="0" max="100"></progress><p id="progress-details"></p></div><div id="status-message"></div><h3>📜 Your Download History</h3><ul id="history-list"></ul>{% else %}<div id="login-section"><h3>Welcome!</h3><p>To save videos to your own Google Drive, you need to log in.</p><a href="/login"><button class="login-btn">Login with Google</button></a></div>{% endif %}</div>{% if 'credentials' in session %}<script>window.onload=function(){loadHistory()};let currentTaskId=null,progressInterval=null,videoInfoCache=null;function setStatus(e,t="info"){const o=document.getElementById("status-message");o.innerHTML=e,o.className=`status status-${t}`,o.style.display=e?"block":"none"}function modeChanged(){const e=document.getElementById("mode").value;document.getElementById("format-select-group").style.display="video"===e||"audio"===e?"block":"none",document.getElementById("transcript-select-group").style.display="transcript"===e?"block":"none",populateSelectors()}function populateSelectors(){if(!videoInfoCache)return;const e=document.getElementById("mode").value,t=document.getElementById("format_id"),o=document.getElementById("transcript_lang");if(t.innerHTML="",o.innerHTML="","video"===e||"audio"===e)videoInfoCache.formats.forEach(o=>{const n="none"===o.vcodec;("audio"===e&&n||"video"===e&&!n)&&(t.innerHTML+=`<option value="${o.format_id}">${o.label}</option>`)});else if("transcript"===e)if(videoInfoCache.transcripts&&Object.keys(videoInfoCache.transcripts).length>0)for(const e in videoInfoCache.transcripts)o.innerHTML+=`<option value="${e}">${videoInfoCache.transcripts[e][0].name||e}</option>`;else o.innerHTML='<option value="">No transcripts found</option>'}async function fetchInfo(){const e=document.getElementById("link").value;if(!e)return void setStatus("Please enter a YouTube URL.","error");setStatus("Fetching video information...","info"),document.getElementById("fetch-btn").disabled=!0,document.getElementById("download-btn").disabled=!0;try{const t=await fetch(`/info?url=${encodeURIComponent(e)}`);if(videoInfoCache=await t.json(),videoInfoCache.error)throw new Error(videoInfoCache.error);document.getElementById("video-title").textContent=videoInfoCache.title,document.getElementById("video-duration").textContent=`Duration: ${videoInfoCache.duration}`,document.getElementById("thumbnail").src=videoInfoCache.thumbnail,document.getElementById("info-preview").style.display="flex",modeChanged(),document.getElementById("download-btn").disabled=!1,setStatus("Information loaded. Ready to download.","success")}catch(e){videoInfoCache=null,setStatus(`Failed to fetch info: ${e.message}`,"error")}finally{document.getElementById("fetch-btn").disabled=!1}}async function startDownload(){let e={link:document.getElementById("link").value,mode:document.getElementById("mode").value,folder:document.getElementById("folder").value};const t=document.getElementById("mode").value;"transcript"===t?e.lang=document.getElementById("transcript_lang").value:e.format_id=document.getElementById("format_id").value;if(!e.lang&&!e.format_id)return void setStatus("No format or language selected.","error");setStatus("Initiating download...","info"),document.getElementById("download-btn").disabled=!0,document.getElementById("fetch-btn").disabled=!0;try{const t=await fetch("/download",{method:"POST",headers:{"Content-Type":"application/json"},body:JSON.stringify(e)}),o=await t.json();if(!o.task_id)throw new Error(o.error||"Failed to start task");currentTaskId=o.task_id,document.getElementById("progress-container").style.display="block",progressInterval=setInterval(pollProgress,1500)}catch(e){setStatus(e.message,"error"),document.getElementById("download-btn").disabled=!1,document.getElementById("fetch-btn").disabled=!1}}async function pollProgress(){if(!currentTaskId)return;try{const e=await fetch(`/progress/${currentTaskId}`);if(!e.ok)throw new Error((await e.json()).message||"Polling failed");const t=await e.json(),o=document.getElementById("progress-container");o.style.display="block";const n=document.getElementById("progress-bar"),s=document.getElementById("progress-status-text"),i=document.getElementById("progress-details");if(!n||!s||!i)return;if(s.textContent=t.message||"Processing...","downloading"===t.status?(n.value=t.progress||0,i.textContent=`(${(t.progress||0).toFixed(2)}%)`):(n.value=100,i.textContent=""),"complete"===t.status||"error"===t.status){if(clearInterval(progressInterval),progressInterval=null,setTimeout(()=>{o.style.display="none"},4e3),"complete"===t.status)setStatus("✅ Download successful!","success");else{const e=t.message.length>300?t.message.substring(0,300)+"...":t.message;setStatus(`❌ Error: ${e}`,"error")}loadHistory(),document.getElementById("download-btn").disabled=!1,document.getElementById("fetch-btn").disabled=!1}}catch(e){clearInterval(progressInterval),setStatus(`Lost connection. ${e.message}`,"error"),document.getElementById("download-btn").disabled=!1,document.getElementById("fetch-btn").disabled=!1}}async function loadHistory(){try{const e=await fetch("/history"),t=await e.json(),o=document.getElementById("history-list");if(o.innerHTML="",""===t||0===t.length)o.innerHTML="<li>No downloads yet.</li>";else t.slice().reverse().forEach(e=>{o.innerHTML+=`<li><b>${e.file_name}</b><br><small>Saved to folder: '${e.drive_folder}'</small></li>`})}catch(e){document.getElementById("history-list").innerHTML="<li>Could not load history.</li>"}}</script>{% endif %}</body></html>
"""

# ----------------------------
# FLASK BACKEND
# ----------------------------
def get_user_drive_service():
    if 'credentials' not in session: return None
    return build('drive', 'v3', credentials=Credentials(**session['credentials']))
@app.route('/')
def root(): return render_template_string(HTML_TEMPLATE)
@app.route('/login')
def login():
    flow = Flow.from_client_secrets_file('client_secrets.json', scopes=SCOPES, redirect_uri=REDIRECT_URI)
    authorization_url, state = flow.authorization_url(access_type='offline', include_granted_scopes='true')
    session['state'] = state
    return redirect(authorization_url)
@app.route('/oauth2callback')
def oauth2callback():
    state = session['state']
    flow = Flow.from_client_secrets_file('client_secrets.json', scopes=SCOPES, state=state, redirect_uri=REDIRECT_URI)
    try:
        flow.fetch_token(authorization_response=request.url)
    except Exception as e:
        print(f"ERROR fetching token: {e}")
        return "Failed to fetch OAuth token. Please try logging in again.", 400
    credentials = flow.credentials
    session['credentials'] = {'token': credentials.token, 'refresh_token': credentials.refresh_token, 'token_uri': credentials.token_uri, 'client_id': credentials.client_id, 'client_secret': credentials.client_secret, 'scopes': credentials.scopes}
    user_info_service = build('oauth2', 'v2', credentials=credentials)
    user_info = user_info_service.userinfo().get().execute()
    session['email'] = user_info.get('email', 'N/A')
    return redirect(url_for('root'))
@app.route('/logout')
def logout(): session.clear(); return redirect(url_for('root'))
@app.route('/info')
def get_info():
    url = request.args.get('url'); ydl_opts = {'quiet': True, 'skip_download': True, 'listsubtitles': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl: info = ydl.extract_info(url, download=False)
    formats = []; all_transcripts = {**info.get('subtitles', {}), **info.get('automatic_captions', {})}
    for f in info.get('formats', []):
        if f.get('acodec') != 'none':
            size_mb=f.get('filesize') or f.get('filesize_approx'); size_str=f"{size_mb/1024/1024:.2f} MB" if size_mb else "N/A"
            formats.append({'format_id':f['format_id'], 'label':f"{f.get('format_note', f.get('resolution', 'Audio'))} ({f.get('ext')}) - {size_str}", 'vcodec':f.get('vcodec', 'none')})
    duration_seconds = info.get('duration', 0)
    duration_formatted = str(timedelta(seconds=int(duration_seconds)))
    return jsonify({'title':info.get('title','Video'),'thumbnail':info.get('thumbnail'),'duration': duration_formatted, 'formats':formats, 'transcripts':all_transcripts})
@app.route('/download', methods=['POST'])
def download():
    if 'credentials' not in session: return jsonify({'error': 'User not authenticated'}), 401
    service = get_user_drive_service()
    if not service: return jsonify({'error': 'Could not create Drive service'}), 500
    task_id = str(uuid.uuid4()); DOWNLOAD_TASKS[task_id] = {'status': 'pending'}
    Thread(target=run_download_task, args=(task_id, request.json, service), daemon=True).start()
    return jsonify({'task_id': task_id})
@app.route('/progress/<task_id>')
def progress(task_id): return jsonify(DOWNLOAD_TASKS.get(task_id, {}))
@app.route('/history')
def history():
    service = get_user_drive_service();
    if not service: return jsonify([]), 401
    files = service.files().list(q=f"name='ytdl_history.json' and 'root' in parents and trashed=false", fields='files(id)').execute().get('files', [])
    if not files: return jsonify([])
    request = service.files().get_media(fileId=files[0]['id']); fh=io.BytesIO(); MediaIoBaseDownload(fh, request).next_chunk()
    try: return jsonify(json.loads(fh.getvalue()))
    except: return jsonify([])

# ----------------------------
# CORE WORKER & HELPERS
# ----------------------------
def get_or_create_folder_id(service, folder_name):
    if not folder_name: return 'root'
    query = f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed=false"
    files = service.files().list(q=query, spaces='drive', fields='files(id)').execute().get('files', [])
    if files: return files[0].get('id')
    return service.files().create(body={'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}, fields='id').execute().get('id')
def update_drive_history(service, new_entry):
    history_filename = "ytdl_history.json"; temp_local_path = f"/tmp/{uuid.uuid4()}.json"
    history = []; file_id = None
    files = service.files().list(q=f"name='{history_filename}' and 'root' in parents and trashed=false", fields='files(id)').execute().get('files', [])
    if files:
        file_id = files[0]['id']; request = service.files().get_media(fileId=file_id); fh = io.BytesIO(); MediaIoBaseDownload(fh, request).next_chunk()
        try: history = json.loads(fh.getvalue().decode('utf-8'))
        except: history = []
    history.append(new_entry)
    with open(temp_local_path, 'w') as f: json.dump(history, f, indent=2)
    media = MediaFileUpload(temp_local_path, mimetype='application/json', resumable=True)
    if file_id: service.files().update(fileId=file_id, media_body=media).execute()
    else: service.files().create(body={'name': history_filename, 'parents': ['root']}, media_body=media).execute()
    os.remove(temp_local_path)
def run_download_task(task_id, data, service):
    try:
        mode, drive_folder_name = data.get('mode'), data.get('folder', 'YouTube Downloads')
        ydl_opts = {'outtmpl': os.path.join(TEMP_DOWNLOAD_DIR, '%(title)s.%(ext)s')}
        if mode == 'transcript':
            ydl_opts.update({'writeautomaticsub': True, 'subtitleslangs': [data.get('lang', 'en')], 'skip_download': True, 'outtmpl': os.path.join(TEMP_DOWNLOAD_DIR, '%(title)s.%(lang)s.%(ext)s')})
        else:
            def progress_hook(d):
                if d['status'] == 'downloading':
                    total = d.get('total_bytes') or d.get('total_bytes_estimate', 0)
                    if total > 0: DOWNLOAD_TASKS[task_id].update({'status':'downloading','message':'Downloading...','progress':(d.get('downloaded_bytes',0)/total)*100})
            ydl_opts.update({'format': data.get('format_id'), 'progress_hooks': [progress_hook]})
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(data['link'], download=True)
            local_path = ydl.prepare_filename(info)
            if mode == 'transcript': local_path = local_path.rsplit('.', 1)[0] + f".{data.get('lang', 'en')}.vtt"
        file_name = os.path.basename(local_path)
        DOWNLOAD_TASKS[task_id].update({'status': 'uploading', 'message': f"Uploading {file_name}..."})
        media = MediaFileUpload(local_path, resumable=True)
        parent_folder_id = get_or_create_folder_id(service, drive_folder_name)
        service.files().create(body={'name': file_name, 'parents': [parent_folder_id]}, media_body=media).execute()
        update_drive_history(service, {"timestamp": time.time(), "link": data['link'], "file_name": file_name, "drive_folder": drive_folder_name})
        os.remove(local_path)
        DOWNLOAD_TASKS[task_id].update({'status': 'complete'})
    except Exception as e:
        err_msg = str(e)
        print(f"ERROR in task {task_id}: {err_msg}")
        DOWNLOAD_TASKS[task_id].update({'status': 'error', 'message': err_msg})

# THE FIX IS HERE: The function for the keep-alive thread.
def _keep_alive_thread():
    """Prints a message to the Colab log every 15 minutes to prevent inactivity timeout."""
    while True:
        # Sleep for 15 minutes (900 seconds)
        time.sleep(900)
        print(f"Keep-alive ping at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ----------------------------
# START THE APP
# ----------------------------
if __name__ == '__main__':
    if NGROK_AUTH_TOKEN == "<YOUR_NGROK_AUTH_TOKEN>" or NGROK_STATIC_DOMAIN == "YOUR_STATIC_DOMAIN_HERE.ngrok-free.app":
        print("🔴 APPLICATION FAILED TO START: Please set your NGROK_AUTH_TOKEN and NGROK_STATIC_DOMAIN variables and restart.")
    elif not os.path.exists('client_secrets.json'):
        print("🔴 APPLICATION FAILED TO START: Please upload your `client_secrets.json` file and restart.")
    else:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("Starting ngrok tunnel with static domain...")
        public_tunnel = ngrok.connect(5000, domain=NGROK_STATIC_DOMAIN)

        # THE FIX IS HERE: Start the keep-alive thread in the background.
        keep_alive = Thread(target=_keep_alive_thread, daemon=True)
        keep_alive.start()

        print("\n====================================================================")
        print("✅ YouTube Downloader is RUNNING!")
        print(f"🌍 PERMANENT Public URL: https://{NGROK_STATIC_DOMAIN}")
        print("🕒 Session is now being kept alive automatically.")
        print("====================================================================\n")
        app.run(port=5000)

🚀 Starting setup...
✅ All packages installed and imported.
✅ `client_secrets.json` found.
Starting ngrok tunnel with static domain...

✅ YouTube Downloader is RUNNING!
🌍 PERMANENT Public URL: https://actively-verified-shrew.ngrok-free.app

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:14:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:14:28] "GET /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:14:45] "GET /oauth2callback?state=Xn6UTJViRM0A5hXKfae7A5rU1W6TbG&code=4/0AUJR-x4_zLVHPQitbgaTvmguVFMC4OCvspSrObE03pQCUr8uAPSD1W7sNgTxW5mL-VaTvQ&scope=email%20https://www.googleapis.com/auth/drive%20https://www.googleapis.com/auth/userinfo.email%20openid&authuser=0&prompt=consent HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:14:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:14:47] "GET /history HTTP/1.1" 200 -


Language Name                  Formats
ab       Abkhazian             vtt, srt, ttml, srv3, srv2, srv1, json3
aa       Afar                  vtt, srt, ttml, srv3, srv2, srv1, json3
af       Afrikaans             vtt, srt, ttml, srv3, srv2, srv1, json3
ak       Akan                  vtt, srt, ttml, srv3, srv2, srv1, json3
sq       Albanian              vtt, srt, ttml, srv3, srv2, srv1, json3
am       Amharic               vtt, srt, ttml, srv3, srv2, srv1, json3
ar       Arabic                vtt, srt, ttml, srv3, srv2, srv1, json3
hy       Armenian              vtt, srt, ttml, srv3, srv2, srv1, json3
as       Assamese              vtt, srt, ttml, srv3, srv2, srv1, json3
ay       Aymara                vtt, srt, ttml, srv3, srv2, srv1, json3
az       Azerbaijani           vtt, srt, ttml, srv3, srv2, srv1, json3
bn       Bangla                vtt, srt, ttml, srv3, srv2, srv1, json3
ba       Bashkir               vtt, srt, ttml, srv3, srv2, srv1, json3
eu       Basque                vtt, sr

INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:14:58] "GET /info?url=https://www.youtube.com/watch?v%3D7j_NE6Pjv-E HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:05] "POST /download HTTP/1.1" 200 -


[youtube] Extracting URL: https://www.youtube.com/watch?v=7j_NE6Pjv-E
[youtube] 7j_NE6Pjv-E: Downloading webpage
[youtube] 7j_NE6Pjv-E: Downloading tv client config
[youtube] 7j_NE6Pjv-E: Downloading tv player API JSON
[youtube] 7j_NE6Pjv-E: Downloading ios player API JSON


...

[download] 100% of   53.39MiB in 00:00:01 at 37.03MiB/s  


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:10] "GET /progress/754cf2d8-dd0e-4cd8-9ece-77ff111c5d3a HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:12] "GET /progress/754cf2d8-dd0e-4cd8-9ece-77ff111c5d3a HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:13] "GET /progress/754cf2d8-dd0e-4cd8-9ece-77ff111c5d3a HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:15] "GET /progress/754cf2d8-dd0e-4cd8-9ece-77ff111c5d3a HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:17] "GET /progress/754cf2d8-dd0e-4cd8-9ece-77ff111c5d3a HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:19] "GET /progress/754cf2d8-dd0e-4cd8-9ece-77ff111c5d3a HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:20] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:29] "GET /logout HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:15:32] 

ERROR fetching token: Scope has changed from "openid https://www.googleapis.com/auth/userinfo.email https://www.googleapis.com/auth/drive" to "https://www.googleapis.com/auth/drive.file openid https://www.googleapis.com/auth/userinfo.email https://www.googleapis.com/auth/drive".


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:16:13] "GET /oauth2callback?state=aZG53ls4QlaoRrD6Mmx1ZSReQY2HQR&code=4/0AUJR-x6p85diZ9wDR7kD1puoA0fqx_iG9phbLOVhW8xmOhW2VV-pgHGyvEpFPF2kjp9yJQ&scope=email+https://www.googleapis.com/auth/drive+openid+https://www.googleapis.com/auth/userinfo.email+https://www.googleapis.com/auth/drive.file&authuser=1&prompt=none HTTP/1.1" 400 -


ERROR fetching token: (invalid_grant) Bad Request
